In [1]:
!pip install pytorch-lightning

In [2]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd 
import pytorch_lightning as L
from torchvision import transforms, models,datasets
import cv2
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader ,random_split,Subset
import matplotlib.pyplot as plt 
import torchvision.models as models
import torch.nn as nn 
import torch.optim as optim 
from torchmetrics import MetricCollection, Accuracy
import torch.nn.functional as F
import torch
import os
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import wandb

In [4]:
wandb.login()
#1b3afb4b4413bbbe24d1004eb7aadcd3a31f1b4a

sweep_config = {
  'name': 'Assignment2_partB',
  'method': 'bayes',
  'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
  'parameters': {
      'epochs': {
            'values': [5,10,15]
        },
        'learning_rate': {
            'values': [0.001,0.0015,0.0001, 0.01]
        },
        'batch_size': {
            'values': [8,16, 32, 64]
        },
        'optimizer':{
              'values': ['adam','nadam','sgd']
        },
        'model':{
            'values':['ResNet50','GoogLeNet','InceptionV3']
        },
        'unfreeze_layers':{
            'values':[0,5,10,20,25]
        }
      
    }
}
sweep_id = wandb.sweep(sweep_config,entity="amar_cs23m011",project="Assignment2-CS6910")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 0gusrwqd
Sweep URL: https://wandb.ai/amar_cs23m011/Assignment2-CS6910/sweeps/0gusrwqd


In [5]:
class root_dataset(Dataset):
    def __init__(self):
        self.dataset1=datasets.ImageFolder(root='/kaggle/input/ineuranet/inaturalist_12K/train')
        l1=int(len(self.dataset1)*0.8)
        train_dataset,val_dataset=random_split(self.dataset1, [int(len(self.dataset1)*0.8),len(self.dataset1)-l1])
        #print(len(train_dataset),len(val_dataset))
        self.train_dataset=train_dataset
        self.val_dataset=val_dataset
    def get_train_data(self):
        return self.train_dataset
    def get_val_data(self):
        return self.val_dataset

In [6]:
class inaturalist_train(Dataset):
    def __init__(self,train_data,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=train_data    
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size 
        
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [7]:
class inaturalist_val(Dataset):
    def __init__(self,val_data,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        #dataset1=datasets.ImageFolder(root='/kaggle/input/neurolist/inaturalist_12K/train')
        self.dataset=val_data
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [8]:
class inaturalist_test(Dataset):
    def __init__(self,model_name):
        if model_name=='InceptionV3':
            self.target_size=(3,299,299)
        else:
            self.target_size=(3,224,224)
        self.dataset=datasets.ImageFolder(root='/kaggle/input/ineuranet/inaturalist_12K/val')
        self.transform = transforms.Compose([
            transforms.Resize(self.target_size[1:]),  # Resize images to target size
            transforms.ToTensor()
        ])#self.target_size = target_size   
    def __getitem__(self,idx):
        image,label=self.dataset[idx]
        image=self.transform(image)
        return image,label
    def __len__(self):
        return len(self.dataset)

In [9]:
class Activation_Function:
    def activation_Function(self,activation_function):
        if activation_function=='relu':
            return F.relu
        if activation_function=='gelu':
            return F.gelu
        if activation_function=='selu':
            return F.selu
        if activation_function=='elu':
            return F.elu

In [10]:
class lightning_pretrained_CNN(L.LightningModule):
    def __init__(self,model_name,unfreeze_layers,optimizer,learning_rate):
        super().__init__()
        self.learning_rate=learning_rate
        self.optimizer=optimizer
        self.model_name=model_name
        if self.model_name=='ResNet50':
            self.model=models.resnet50(pretrained=True)
            #print('hi')
        if self.model_name=='GoogLeNet':
            self.model=models.googlenet(pretrained=True)
        if self.model_name=='InceptionV3':
            self.model=models.inception_v3(pretrained=True,transform_input=True)
        freeze_index=0
        for param in self.model.parameters():
            if freeze_index< (len(list(self.model.parameters())) - (unfreeze_layers + 2)):
                param.requires_grad = False
            else:
                break
            freeze_index=freeze_index+1
        num_feature=self.model.fc.in_features
        self.model.fc=nn.Linear(num_feature,10)
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        inputs,labels=batch
        output=self.forward(inputs)
        #print(type(output.logits))
        #print(output.shape)
        #print(output.logits)
        if self.model_name=='InceptionV3':
            _,preds = torch.max(output.logits, dim=1)
            loss=F.cross_entropy(output.logits,labels)
        #train_acc = torch.mean(preds == labels)
        #print(pred.shape)
            self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        else:
            _,preds = torch.max(output, dim=1)
            loss=F.cross_entropy(output,labels)
            self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    def configure_optimizers(self):
        if self.optimizer=='adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='nadam':
            optimizer = torch.optim.NAdam(self.parameters(), lr=self.learning_rate)
            return optimizer
        if self.optimizer=='sgd':
            optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
            return optimizer
        
    def validation_step(self,batch,batch_idx):
        x, y = batch
        y_pred = self.forward(x)
        val_loss = F.cross_entropy(y_pred, y)
        
        # Compute validation accuracy
        _, predicted = torch.max(y_pred, 1)
        val_acc = torch.sum(predicted == y).item() / y.size(0)
        
        self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', val_acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
        return val_loss
        
    
    def test_step(self, batch, batch_idx):
        x,y=batch
        pred=self.forward(x)
        loss=F.cross_entropy(pred,y)
        _, predicted = torch.max(pred.data, 1)
        accuracy = torch.sum(predicted == y).item() / y.size(0)
        #print(predicted,accuracy)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_accuracy", accuracy, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"test_loss": loss}
    

In [11]:
def main(config=None):
    with wandb.init(config=config, ):
        config=wandb.config
        wandb.run.name='bs-'+str(config.batch_size)+'-lr-'+ str(config.learning_rate)+'-ep-'+str(config.epochs)+ '-op-'+str(config.optimizer)+'-mn-'+str(config.model)+'-ul-'+str(config.unfreeze_layers)
        model_name=config.model
        root_obj=root_dataset()
        train_data=root_obj.get_train_data()
        val_data=root_obj.get_val_data()
        dataset1=inaturalist_train(train_data,model_name)
        dataset2=inaturalist_val(val_data,model_name)
        dataset3=inaturalist_test(model_name)
        b_size=config.batch_size
        unfreeze_layers=config.unfreeze_layers
        optimizer=config.optimizer
        epoch=config.epochs
        learning_rate=config.learning_rate
        wandb_logger = WandbLogger(project='amar_cs23m011', entity='Assignment2-CS6910')
        dataloader=DataLoader(dataset=dataset1,batch_size=b_size,shuffle=True,num_workers=2)
        val_dataloader=DataLoader(dataset=dataset2,batch_size=b_size,shuffle=False,num_workers=2)
        model=lightning_pretrained_CNN(model_name,unfreeze_layers,'adam',0.0015)
        trainer = L.Trainer(accelerator='auto',devices="auto",max_epochs=epoch,logger=wandb_logger)
        trainer.fit(model,dataloader,val_dataloader)
        test_dataloader=DataLoader(dataset=dataset3,batch_size=8,shuffle=False,num_workers=1)
        trainer.test(dataloaders=test_dataloader)

        

In [ ]:
if  __name__ =="__main__":
    wandb.agent('4pa8pa5a', main, count=60)
    #main()

wandb: Agent Starting Run: 27bwbiff with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0001
wandb: 	model: InceptionV3
wandb: 	optimizer: sgd
wandb: 	unfreeze_layers: 25
wandb: Currently logged in as: kumarsharmaamar512 (amar_cs23m011). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 151MB/s]  
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogg

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

epoch,▁▁▂▂▂▂▃▃▄▄▅▅▅▅▆▆▇▇▇▇█
test_accuracy,▁
test_loss,▁
train_loss,█▄▃▂▁▁▁▁▁▁
trainer/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
val_acc,▆▅█▄▄▆▂▁▆▂
val_loss,▁▂▂▅▅▆▇█▆▇
epoch,10
test_accuracy,0.78
test_loss,1.17138
train_loss,0.0499


wandb: Agent Starting Run: nwusa1pq with config:
wandb: 	batch_size: 64
wandb: 	epochs: 15
wandb: 	learning_rate: 0.0015
wandb: 	model: InceptionV3
wandb: 	optimizer: sgd
wandb: 	unfreeze_layers: 5


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]